<a href="https://colab.research.google.com/github/Ewha-SkyEwha/SkyEwha/blob/main/whisper_keyword_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/openai/whisper.git -q
!pip install ffmpeg-python -q
import whisper
import os

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#from google.colab import files
#uploaded = files.upload()

In [4]:
file_path = "/content/drive/MyDrive/Test1.MP4"

In [5]:
# Whisper 모델 로드 및 처리
model = whisper.load_model("medium")  # base, small, medium, large 가능
result = model.transcribe(file_path, language="Korean")
text= result["text"]

100%|█████████████████████████████████████| 1.42G/1.42G [00:17<00:00, 85.4MiB/s]


In [6]:
# 텍스트 출력
print("\n🎯 추출된 텍스트:\n")
print(text)


🎯 추출된 텍스트:

 대전 당일치기 데이트 코스 바로 알려드릴게요. 먼저 고속터미널에서 2시간 버스를 타고 대전에 도착하자마자 새로 생긴 성심당 샌드위치 정거장에 가줍니다. 무거운 빵 들고 돌아다니면 힘드니까 걸어서 1분 거리인 빵봉환소에 가서 빵을 맡겨주고 핏맛집 코스 가득한 토박이에서 오징어주물럭을 먹어줬는데 가격도 저렴하고 엄청 맛있었어요. 밥 먹고 카페에 가서 커피 수혈을 하면서 좀 쉬다가 버스를 30분 타면 국립중앙과학관에 도착하는데 입장비가 무료에다 성인들도 재밌게 즐겨주시고요. 재밌게 즐길 수 있는 체험이 진짜 많아서 완전 강추예요. 저녁은 유명한 오씨 칼국수에서 든든하게 뜨끈한 칼국수까지 먹어주면 완전 알차게 놀고 온 대전 데이트 코스 끝


In [7]:
# env 파일 업로드
from google.colab import files
uploaded = files.upload()

Saving .env to .env (1)


In [8]:
!pip install python-dotenv
import os
from dotenv import load_dotenv

# .env 파일 불러오기
load_dotenv()

# 환경변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

In [9]:
# 텍스트에서 키워드 추출
!pip install openai
import openai

In [10]:
client = openai.OpenAI(api_key = api_key)

def extract_hashtags(text):
    prompt = f"""
당신은 콘텐츠 마케팅 전문가입니다.

다음 한국어 텍스트 내용을 기반으로, 여행 유튜브 영상 설명란에 넣을 수 있는 **해시태그**들을 생성해주세요.
- 각 해시태그는 1~3단어로 구성해주세요.
- "#"을 붙인 형식으로 리스트 형태로 반환해주세요.
- 너무 일반적인 단어 대신 이 영상의 주제와 관련된 구체적인 키워드를 사용해주세요.
- 한국인 여행 유튜버를 위한 서비스임을 고려해주세요.
- 한국어 텍스트 내용을 기반으로 한국 국내 여행이면 해시태그에 '(해당지역)+여행'이라는 키워드는 필수로 포함해주세요.
- 한국어 텍스트 내용을 기반으로 한국이 아닌 해외 여행이면 해시태그에 '(해당나라)+여행'과 '(해당도시)+여행'이라는 키워드는 필수로 포함해주세요.
- 만약 키워드를 리스트 형태로 반환하다가 중간에 응답 길이 제한에 걸리면, 해당 (#키워드)는 삭제하고 출력해주세요.

텍스트:
\"\"\"{text}\"\"\"

출력 예시:
#웨딩촬영
#제주도여행
#파리여행
#이탈리아여행
#여름휴가
"""

    response = client.chat.completions.create(
        model="gpt-4",  # 또는 gpt-3.5-turbo
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.4, #창의성 조절 (0에 가까울수록 일정하고 정확함, 1에 가까울수록 창의적)
        max_tokens=200 #응답 길이 제한
    )

    hashtags = response.choices[0].message.content.strip()
    return hashtags

In [11]:
keywords = extract_hashtags(text)
print("키워드 추출 결과:\n", keywords)

키워드 추출 결과:
 #대전여행
#당일치기데이트
#성심당샌드위치
#토박이오징어주물럭
#국립중앙과학관
#오씨칼국수
#데이트코스
#맛집투어
#카페투어
#빵봉환소
#핏맛집
#커피수혈
#무료입장
#체험여행
#알찬여행


In [12]:
# prompt = f"""
# 당신은 여행 콘텐츠에 특화된 콘텐츠 마케팅 전문가입니다.

# 다음 한국어 텍스트를 바탕으로 **유튜브 여행 영상 설명란에 넣을 해시태그**들을 생성해주세요.
# 음성 인식 오류로 일부 단어가 잘못 표기되었을 수 있습니다. 문맥상 어색한 단어나 잘못된 고유명사는 의미를 유추해 정확하게 정정한 후 해시태그를 생성해주세요.

# 🎯 해시태그 생성 기준:
# - 해시태그는 **형식**: `#단어`, 단어는 **1~3개**로 구성
# - **너무 일반적인 단어** 대신 **영상의 구체적 내용, 장소, 활동, 음식, 교통 수단 등**을 반영
# - 한국 국내 여행이면 **(해당지역)+여행** 키워드를 **반드시 포함**
# - 해외 여행이면 **(나라+여행)**, **(도시+여행)** 키워드 **반드시 포함**
# - 해시태그는 **중복되거나 의미가 겹치는 표현** 없이 **최대 20개 이하**로 추려주세요.
# - 생성된 해시태그는 아래 기준에 따라 **중요도 순으로 정렬**해주세요:
#   1. 사용자 검색 가능성이 높은 해시태그 (#지역+여행, #맛집 등)을 우선 배치
#   2. 영상 텍스트에서 자주 등장하거나 중심적인 내용일수록 우선 배치
#   3. 장소명, 브랜드명 등 고유명사는 상단에 배치
#   4. 너무 일반적인 단어보다는 영상 특화 키워드를 우선적으로 반영
#   5. 의미가 비슷한 키워드가 있을 경우, 더 구체적이고 검색성이 높은 쪽만 남겨주세요



# - 해시태그를 **카테고리별로 그룹화해서** 출력해주세요 (예: 📍장소, 🍜음식, ☕카페, 🚌교통, 💑데이트 등)
# 💡 출력 형식 예시:
# 📍 장소
# #제주도여행
# #국립중앙과학관

# 🍜 음식
# #성심당샌드위치
# #오씨칼국수

# 💑 데이트
# #대전데이트
# #당일치기코스

# 🚌 교통
# #버스여행
# #고속터미널

# 📝 텍스트:
# \"\"\"{text}\"\"\"

# 출력 예시:
# #웨딩촬영
# #제주도여행
# #파리여행
# #이탈리아여행
# #여름휴가
# """
